In [5]:
using MathOptInterface
using Ipopt
using Test
using LinearAlgebra
using DiffOpt

const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities;

In [6]:
# non-homogeneous quadratic objective
#    minimize 2 x^2 + y^2 + xy + x + y
#       s.t.  x, y >= 0
#             x + y = 1

model = MOI.instantiate(Ipopt.Optimizer, with_bridge_type=Float64)
x = MOI.add_variable(model)
y = MOI.add_variable(model)

c1 = MOI.add_constraint(
    model,
    MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([1.0,1.0], [x,y]), 0.0),
    MOI.EqualTo(1.0)
)

vc1 = MOI.add_constraint(
    model, 
    MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([-1.0,0.0], [x,y]), 0.0), 
    MOI.LessThan(0.0)
)
@test vc1.value == x.value

vc2 = MOI.add_constraint(
    model,
    MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.([0.0,-1.0], [x,y]), 0.0), 
    MOI.LessThan(0.0)
)
@test vc2.value == y.value


MOI.set(model, MOI.ObjectiveSense(), MOI.MIN_SENSE)
obj = MOI.ScalarQuadraticFunction(
    MOI.ScalarAffineTerm.([1.0, 1.0], [x, y]),
    MOI.ScalarQuadraticTerm.([4.0, 2.0, 1.0], [x, y, x], [x, y, y]),
    0.0
)
MOI.set(model, MOI.ObjectiveFunction{MOI.ScalarQuadraticFunction{Float64}}(), obj)

In [7]:
dm = diff_model(model)

#1 (generic function with 1 method)

In [8]:
z, λ, ν = dm.forward()

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        2

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

([0.25, 0.75], [-1.00978e-8, -3.33506e-9], [2.75])

In [9]:
atol=1e-4 
rtol=1e-4

@test z ≈ [0.25, 0.75] atol=atol rtol=rtol
@test λ ≈ [0.0, 0.0]   atol=atol rtol=rtol
@test ν ≈ [11/4]       atol=atol rtol=rtol

Test Passed

In [10]:
nz = 2
nineq = 2
neq = 1

Q = [4. 1.; 1. 2.]
q = [1.; 1.]
G = [-1. 0.0; 0.0 -1.0]
h = [0.0; 0.0]
A = [1.0 1.0;]
b = [1.0]

1-element Array{Float64,1}:
 1.0

In [11]:
# lhs = DiffOpt.create_LHS_matrix(z, λ, Q, G, h)
lhs = [Q    G'*Diagonal(λ)         A';
       G    Diagonal((G * z) - h)  zeros(nineq, neq);
       A    zeros(neq, nineq)      zeros(neq, neq)]

5×5 Array{Float64,2}:
  4.0   1.0   1.00978e-8  -0.0         1.0
  1.0   2.0  -0.0          3.33506e-9  1.0
 -1.0   0.0  -0.25         0.0         0.0
  0.0  -1.0   0.0         -0.75        0.0
  1.0   1.0   0.0          0.0         0.0

In [12]:
rhs = [[1.3; 0.5];
        zeros(neq+nineq,1)]

5×1 Array{Float64,2}:
 1.3
 0.5
 0.0
 0.0
 0.0

In [13]:
sol = -(lhs \ rhs)

5×1 Array{Float64,2}:
 -0.2000000022418942 
  0.20000000224189418
  0.8000000035574014 
 -0.2666666702569895 
 -0.7000000013525438 

In [14]:
dz = sol[1:2]
dλ = sol[3:4]
dν = sol[5]

-0.7000000013525438

In [15]:
dl_dQ = 0.5*(dz*z' + z*dz')

2×2 Array{Float64,2}:
 -0.05  -0.05
 -0.05   0.15

In [16]:
dl_dq = dz

2-element Array{Float64,1}:
 -0.2000000022418942 
  0.20000000224189418

In [17]:
dl_dG = Diagonal(λ)*dλ*z' - λ*dz' 

2×2 Array{Float64,2}:
 -4.03911e-9   -4.03911e-9
 -4.44675e-10   1.33403e-9

In [18]:
dl_dh = -Diagonal(λ) * dλ

2-element Array{Float64,1}:
  8.078226395049665e-9 
 -8.893504039565539e-10

In [19]:
dl_dA = dν*z' - ν*dz'

1×2 Array{Float64,2}:
 0.375  -1.075

In [20]:
dl_db = -dν

0.7000000013525438